<a href="https://colab.research.google.com/github/deekshithpattipaka/My_Repo/blob/main/SITHAFAL_TASK1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install langchain
!pip install PyPDF2
!pip install openai
!pip install faiss-cpu


In [10]:
!pip install langchain-community

#Importing Required Libraries

In [11]:
import os
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import faiss
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 1.Data Ingestion

In [23]:
def extract_text_from_pdf(pdf_path):
    #Extracts text from a PDF file.
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

def split_text_into_chunks(text, chunk_size=500, overlap=50):
    #Splits text into smaller chunks
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunks.append(text[i : i + chunk_size])
    return chunks

# 2. Vector Database and Embedding The chunks

In [13]:
def create_vector_store(chunks, model_name):
    #Converts text chunks into vector embeddings and stores them in Faiss
    model = SentenceTransformer(model_name)
    embeddings = model.encode(chunks)

    # Initializing the index
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index, embeddings, chunks

def query_vector_store(index, chunks, query, model_name, top_k=5):
    # Queries the vector store to retrieve relevant chunks
    model = SentenceTransformer(model_name)
    query_vector = model.encode([query])
    distances, indices = index.search(query_vector, top_k)
    return [chunks[i] for i in indices[0]]

# 3. LLM(Large Language Model) for Response Generation

In [14]:
def generate_response(retrieved_chunks, query, model_name="google/flan-t5-large"):
    # Generating the  response using an open-source LLM
    context = "\n".join(retrieved_chunks)
    prompt = f"Answer the question based on the following context:\n\n{context}\n\nQuestion: {query}"

    # Loading Hugging Face model for text generation
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
    outputs = model.generate(
        inputs.input_ids, max_length=300, num_return_sequences=1, temperature=0.3
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Workflow

In [20]:

pdf_path = input("Enter the PDF File Path")
print(pdf_path)
embedding_model = "all-MiniLM-L6-v2"

Enter the PDF File Path/content/SITHAFAL_DOCUMENT.pdf
/content/SITHAFAL_DOCUMENT.pdf


### Step 1: Extracting text from PDF and splitting  into chunks

In [16]:
print("Extracting text from PDF...")
text = extract_text_from_pdf(pdf_path)
chunks = split_text_into_chunks(text)

Extracting text from PDF...


### Step 2: Creating vector store

In [17]:
print("Creating vector store...")
vector_store, embeddings, chunks = create_vector_store(chunks, embedding_model)
print(vector_store)

Creating vector store...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7a8e037bbed0> >


### Step 3: Handling the User Query

In [21]:
query =input("Enter The Query:")
print(query)  # Example query
print("Retrieving relevant chunks...")
retrieved_chunks = query_vector_store(vector_store, chunks, query, embedding_model)

Enter The Query:Example from Everyday Life?
Example from Everyday Life?
Retrieving relevant chunks...


### Step 4: Generating The Response

In [22]:
print("Generating response...")
response = generate_response(retrieved_chunks, query)
print("Response:", response)

Generating response...


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Response: 19% 10% 15% 5%26%25%Family Budget of $31,000 Other Recreation Transportation Clothing housing Food
